In [1]:
import scanpy as sc
import matplotlib as plt
import anndata
import numpy as np

In [2]:
complete = sc.read_h5ad('/hpcwork/rwth1209/data/merfish/test_imputation/heart_high_counts.h5ad')
#scref_all = sc.read_h5ad("/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/data_preprocessing/scrna_ref_all_raw_array.h5ad")

In [3]:
complete

AnnData object with n_obs × n_vars = 180489 × 500
    obs: 'fov', 'volume', 'min_x', 'min_y', 'max_x', 'max_y', 'anisotropy', 'transcript_count', 'perimeter_area_ratio', 'solidity', 'CX43_raw', 'CX43_high_pass', 'Cellbound2_raw', 'Cellbound2_high_pass', 'Cellbound1_raw', 'Cellbound1_high_pass', 'Cellbound3_raw', 'Cellbound3_high_pass', 'DES_raw', 'DES_high_pass', 'VIM_raw', 'VIM_high_pass', 'DAPI_raw', 'DAPI_high_pass', 'NCAM1_raw', 'NCAM1_high_pass', 'MPO_raw', 'MPO_high_pass', 'COL1A_raw', 'COL1A_high_pass', 'PolyT_raw', 'PolyT_high_pass', 'region', 'COLIII_raw', 'COLIII_high_pass', 'Beta-3-Tubulin_raw', 'Beta-3-Tubulin_high_pass', 'COL3_raw', 'COL3_high_pass', 'dataset', 'n_counts', 'n_genes', 'tacco', 'tacco_score', '_scvi_batch', '_scvi_labels', 'leiden_R', 'fibro_score', 'myofibro_score', 'Fibroblast_Type', 'Fibroblast_Leiden', 'Preliminary CT', 'Preliminary_CT', 'peri_score', 'leiden_0.2', 'leiden_0.5', 'leiden_1', 'leiden_1.5', 'cell_type', 'cleanup_0', 'cardio_score', 'adipo_s

###### Super specific genes: CCL21,MMRN1,PRF1,TBX1,PRF1
###### Medium specific genes: NEXN,CACNA1D,PPP2R2B,BASP1,TM4SF1
###### Low specific genes: RNF19A,IMMP2L,NFKB1,CCDC141,SMC4
###### three genes that are in spatial but not in scref (otherwise gives an error) not_in_ref = ['MARCHF1','CYRIB','PRANCR']

In [4]:
genes_to_remove = ['CCL21', 'MMRN1', 'PRF1', 'TBX1', 'PRF1',
                    'NEXN', 'CACNA1D', 'PPP2R2B', 'BASP1', 'TM4SF1',
                      'RNF19A', 'IMMP2L', 'NFKB1', 'CCDC141', 'SMC4',
                      'MARCHF1','CYRIB','PRANCR']
#genes_to_remove = ['MARCHF1','CYRIB','PRANCR']
#not_in_ref = ['MARCHF1','CYRIB','PRANCR']
#genes_complete = list(complete.var_names)
#all_genes_ref = list(scref_all.var_names)


In [6]:
len(genes_complete)

500

In [7]:
#save all genes from complete to .txt file
with open('500_gene_list.txt', 'w') as f:
    # Write each gene to the file
    for gene in genes_complete:
        f.write(f"{gene}\n")

# First prepeare spatial data (removing genes that are going to be tested and that are not in the ref dataset)

In [5]:
# .X norm to raw
complete.X.max()

330.0

In [6]:
X_array = (complete.layers['counts']).toarray() 

In [7]:
complete.X = X_array

In [8]:
complete.X.max()

330.0

In [19]:
# for gene in genes_complete:
#     if gene not in all_genes_ref:
#         print(gene)

MARCHF1
CYRIB
PRANCR


In [9]:
genes_to_keep = [gene for gene in complete.var_names if gene not in genes_to_remove]

withhelded = complete[:, genes_to_keep]

In [10]:
len(genes_to_keep)

483

In [11]:
withhelded

View of AnnData object with n_obs × n_vars = 180489 × 483
    obs: 'fov', 'volume', 'min_x', 'min_y', 'max_x', 'max_y', 'anisotropy', 'transcript_count', 'perimeter_area_ratio', 'solidity', 'CX43_raw', 'CX43_high_pass', 'Cellbound2_raw', 'Cellbound2_high_pass', 'Cellbound1_raw', 'Cellbound1_high_pass', 'Cellbound3_raw', 'Cellbound3_high_pass', 'DES_raw', 'DES_high_pass', 'VIM_raw', 'VIM_high_pass', 'DAPI_raw', 'DAPI_high_pass', 'NCAM1_raw', 'NCAM1_high_pass', 'MPO_raw', 'MPO_high_pass', 'COL1A_raw', 'COL1A_high_pass', 'PolyT_raw', 'PolyT_high_pass', 'region', 'COLIII_raw', 'COLIII_high_pass', 'Beta-3-Tubulin_raw', 'Beta-3-Tubulin_high_pass', 'COL3_raw', 'COL3_high_pass', 'dataset', 'n_counts', 'n_genes', 'tacco', 'tacco_score', '_scvi_batch', '_scvi_labels', 'leiden_R', 'fibro_score', 'myofibro_score', 'Fibroblast_Type', 'Fibroblast_Leiden', 'Preliminary CT', 'Preliminary_CT', 'peri_score', 'leiden_0.2', 'leiden_0.5', 'leiden_1', 'leiden_1.5', 'cell_type', 'cleanup_0', 'cardio_score', 

In [13]:
withhelded.X.max()

array(330., dtype=float32)

In [12]:
withhelded.X

ArrayView([[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
withhelded.write('withhelded_high_counts_array_raw_483_genes.h5ad')

# Second prepare scref dataset (array, raw counts,)
# HVG 3000

In [28]:
scref = sc.read_h5ad("/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/ref/snRNA_teichman_atlas_normalized.h5ad")
scref_raw = sc.read_h5ad("/work/rwth1209/dana_projects/test_new_imputation_tools/enVI/ref/snRNA_teichman_atlas_raw.h5ad")

/home/xu262605/anaconda3/envs/spatial_data/lib/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [29]:
sc.pp.highly_variable_genes(scref, n_top_genes = 3000)

/home/xu262605/anaconda3/envs/spatial_data/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]


In [35]:
#del scref.var['gene_name_multiome-2-original']

In [36]:
scref.var

,gene_id,highly_variable,means,dispersions,dispersions_norm
gene_name-new,,,,,
MIR1302-2HG,ENSG00000243485,False,5.424499e-04,1.904405,0.162199
FAM138A,ENSG00000237613,False,1.000000e-12,NaN,NaN
OR4F5,ENSG00000186092,False,1.359877e-04,1.460643,-0.863429
AL627309.1,ENSG00000238009,False,4.132853e-02,1.850842,0.038404
AL627309.3,ENSG00000239945,False,5.499083e-06,0.346018,-3.439563
...,...,...,...,...,...
AC233755.2,ENSG00000277856,False,1.621396e-05,1.177938,-1.516820
AC233755.1,ENSG00000275063,False,1.395225e-05,1.277081,-1.287678
AC240274.1,ENSG00000271254,False,3.349808e-02,1.911611,0.178854


In [37]:
scref.X.max()

9.030837

In [39]:
scref.layers['counts_norm'] = scref.X

In [41]:
scref.X = scref_raw.X

In [42]:
scref.X.max()

17441.0

In [43]:
scref.X

<257029x32732 sparse matrix of type '<class 'numpy.float32'>'
	with 372161202 stored elements in Compressed Sparse Row format>

In [44]:
array_X = (scref.X).toarray() 

In [45]:
array_X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [46]:
del scref.X

In [47]:
scref.X = array_X

In [48]:
#filter out genes; only keep hvg and which are in st data
import numpy as np

highly_variable_genes = scref.var['highly_variable']

genes_to_keep = list(withhelded.var_names)
genes_to_keep_set = set(genes_to_keep)

combined_genes = np.union1d(np.where(highly_variable_genes)[0], np.where(scref.var_names.isin(genes_to_keep_set))[0])
scref = scref[:, combined_genes]

In [49]:
scref

View of AnnData object with n_obs × n_vars = 257029 × 3180
    obs: 'sangerID', 'donor', 'donor_type', 'region', 'age', 'gender', 'facility', 'cell_or_nuclei', 'modality', 'kit_10x', 'flushed', 'cell_type', 'cell_state', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'scrublet_score'
    var: 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'age_colors', 'cell_or_nuclei_colors', 'cell_state_colors', 'cell_type_colors', 'donor_colors', 'donor_type_colors', 'facility_colors', 'flushed_colors', 'gender_colors', 'kit_10x_colors', 'leiden', 'log1p', 'modality_colors', 'neighbors', 'original_or_new_colors', 'region_colors', 'region_finest_colors', 'scANVI_predictions_colors', 'umap', 'hvg'
    obsm: 'X_umap'
    layers: 'counts_norm'
    obsp: 'connectivities', 'distances'

In [50]:
scref.X

ArrayView([[  0.,   1.,   0., ..., 175., 222.,   0.],
           [  0.,   4.,   0., ..., 185., 142.,   0.],
           [  0.,   2.,   0., ...,  53.,  70.,   0.],
           ...,
           [  0.,   0.,   0., ...,   0.,   0.,   0.],
           [  0.,   0.,   0., ...,   0.,   0.,   0.],
           [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32)

In [51]:
scref.X.max()

array(17441., dtype=float32)

In [52]:
scref.write('scref_3000hvg.h5ad')